In [1]:
import numpy as np  
from PIL import Image  
import cv2  
import matplotlib.pyplot as plt 
import pickle  
from matplotlib import style
import time  
import math

from func import *

style.use("ggplot")  # setting style

In [2]:
SIZE = 20

EPISODES = 25000
MOVE_PENALTY = 1  
NFZ_PENALTY = 300  
POI_REWARD = 25
epsilon = 0.5 
EPS_DECAY = 0.9999 
SHOW_EVERY = 1000
EPISODE_STEPS=200

# start_q_table = "qtable-1613141471.pickle"
start_q_table = None

LEARNING_RATE = 0.1
DISCOUNT = 0.95

ACTIONS = 4
AGENTS = 3
PoIs = 20
NFZs = 5

Environment_grid = np.zeros((SIZE, SIZE, 3), dtype=np.uint8) #size*size grid with 3 values at each point to represent color
Occupied = np.zeros((SIZE, SIZE), dtype=np.uint8)

UAV_P = 1  
POI_P = 2  
NFZ_P = 3
CUR_P = 4
VSTD_P = 5



d = {1: (255, 175, 0),  # blueish color
     2: (0, 255, 0),  # green
     3: (0, 0, 255),   # red
     4: (0, 120, 0),   # light green
     5: (255,255,255)}  # white

In [3]:
def initialize_reg(AGENTS, PoIs, NFZs):
    Agent_list = []
    POI_list = []
    NFZ_list = []
    for i in range(AGENTS):
        agent = OBJECT()
        Agent_list.append(agent)
    
    for i in range(PoIs):
        poi = OBJECT()
        POI_list.append(poi)

    for i in range(NFZs):
        nfz = OBJECT()
        NFZ_list.append(nfz)
        
    for i in range(AGENTS):
        curr_obj = OBJECT()
        curr_obj.x = -1
        curr_obj.y = -1
        current_PoI_list.append(curr_obj)

    return Agent_list, POI_list, NFZ_list, current_PoI_list

In [4]:
class OBJECT:
    def __init__(self):
        x=0
        y=0
        while(1):
            a = np.random.randint(0, SIZE)
            b = np.random.randint(0, SIZE)
            if (a,b) in Uniqe_points:
                continue
            else:
                Uniqe_points.append((a,b))
                x=a
                y=b
                break
        self.x = x
        self.y = y
        self.goals = []
        self.visited = []
        self.val=-1
        self.idx=-1
    def __str__(self):
        return f"{self.x}, {self.y}"
    def __sub__(self, other):
        return (self.x-other.x, self.y-other.y)
    
    def action(self, choice):
        '''
        Gives us 8 total movement options. (0,1,2,3,4,5,6,7)
        '''
        if choice == 0:
            self.move(x=1, y=1)
        elif choice == 1:
            self.move(x=-1, y=-1)
        elif choice == 2:
            self.move(x=-1, y=1)
        elif choice == 3:
            self.move(x=1, y=-1)
        elif choice == 4:
            self.move(x=0, y=-1)
        elif choice == 5:
            self.move(x=0, y=1)
        elif choice == 6:
            self.move(x=1, y=0)
        elif choice == 7:
            self.move(x=-1, y=0)
            
    def move(self, x=False, y=False):

        # If no value for x, move randomly
        if not x:
            self.x += np.random.randint(-1, 2)
        else:
            self.x += x

        # If no value for y, move randomly
        if not y:
            self.y += np.random.randint(-1, 2)
        else:
            self.y += y


        # If we are out of bounds, fix!
        if self.x < 0:
            self.x = 0
        elif self.x > SIZE-1:
            self.x = SIZE-1
        if self.y < 0:
            self.y = 0
        elif self.y > SIZE-1:
            self.y = SIZE-1

In [5]:
if start_q_table is None:
    # initialize the q-table#
    q_table = {}
    for i in range(-SIZE+1, SIZE):
        for ii in range(-SIZE+1, SIZE):
            for iii in range(-SIZE+1, SIZE):
                    for iiii in range(-SIZE+1, SIZE):
                        q_table[((i, ii), (iii, iiii))] = [np.random.uniform(-5, 0) for i in range(8)]
else:
    with open(start_q_table, "rb") as f:
        q_table = pickle.load(f)

In [6]:
print(len(q_table))
print(q_table[((0, 1), (3, 9))])

2313441
[-2.0556532068363516, -0.8520924657501672, -4.973995325305959, -4.141629170599288, -0.407322644900292, -0.22035512218502884, -0.7542809670431092, -0.209869451209431]


In [7]:
episode_rewards = []

MAX_VISITED = 0

show_it = []
rewards = []

for episode in range(EPISODES):

    Uniqe_points = []
    Agent_list = []
    POI_list = []
    NFZ_list = []
    current_PoI_list =[]

    Agent_list, POI_list, NFZ_list, current_PoI_list = initialize_reg(AGENTS, PoIs, NFZs)

    paths_of_all_agents =[]

    for i in range(len(Agent_list)):
        a=[]
        paths_of_all_agents.append(a)

    for j in range(len(POI_list)):
        i = j%len(Agent_list)
        Agent_list[i].goals.append((POI_list[j].x,POI_list[j].y))
        Agent_list[i].visited.append(0)


    for i in range(len(Agent_list)):

        assign_weights(Agent_list[i])


        if Agent_list[i].goals:
            val, idx = min((val, idx) for (idx, val) in enumerate(Agent_list[i].visited))

            Agent_list[i].val = val
            Agent_list[i].idx = idx

            current_PoI_list[i].x = Agent_list[i].goals[idx][0]
            current_PoI_list[i].y = Agent_list[i].goals[idx][1]

   
    if episode % SHOW_EVERY == 0:
        print("###########################################################\n\n")
        print(f"on #{episode}, epsilon is {epsilon}")
        print(f"{SHOW_EVERY} ep mean: {np.mean(episode_rewards[-SHOW_EVERY:])}")
        show = True
    else:
        show = False
        
    episode_reward = 0
    
    reward_each_agent =[0]*len(Agent_list)

    for i in range(EPISODE_STEPS):
        
        ####################################################
        for agnt_i in range(len(Agent_list)):
            
            paths_of_all_agents[agnt_i].append((Agent_list[agnt_i].x,Agent_list[agnt_i].y))
            
            if current_PoI_list[agnt_i].x != -1:
                obs = (Agent_list[agnt_i]-current_PoI_list[agnt_i], NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE))

                if np.random.random() > epsilon:
                    # GET THE ACTION
                    action = np.argmax(q_table[obs])
                else:
                    action = np.random.randint(0, 7)

                # Taking selected action
                Agent_list[agnt_i].action(action)
                
                

                if Agent_list[agnt_i].x == NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE)[0] and Agent_list[agnt_i].y == NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE)[1]:
                    reward_each_agent[agnt_i] = -NFZ_PENALTY
                elif Agent_list[agnt_i].x == current_PoI_list[agnt_i].x and Agent_list[agnt_i].y == current_PoI_list[agnt_i].y:
                    reward_each_agent[agnt_i] = POI_REWARD
                else:
                    reward_each_agent[agnt_i] = -MOVE_PENALTY


                new_obs = (Agent_list[agnt_i]-current_PoI_list[agnt_i], NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE))  # new observation
                max_future_q = np.max(q_table[new_obs])  # max Q value for this new obs
                current_q = q_table[obs][action]  # current Q for our chosen action

                if reward_each_agent[agnt_i] == POI_REWARD:
                    new_q = POI_REWARD
                elif reward_each_agent[agnt_i] == -NFZ_PENALTY:
                    new_q = -NFZ_PENALTY
                else:
                    new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward_each_agent[agnt_i] + DISCOUNT * max_future_q)


                q_table[obs][action] = new_q
            else:
                break
        
        #############################################################
        
        if show:
            env = np.zeros((SIZE, SIZE, 3), dtype=np.uint8)  # starts an rbg of our size
            
            
            
            for i in range(len(POI_list)):
                env[POI_list[i].x][POI_list[i].y] = d[POI_P] # sets the Point of interest tile to green color
            
            for agnt_i in range(len(Agent_list)):
                if current_PoI_list[agnt_i].x != -1:
                    env[current_PoI_list[agnt_i].x][current_PoI_list[agnt_i].y] = d[CUR_P]  # sets the current Point of interest tile to light green color
                
                for i,v in enumerate(Agent_list[agnt_i].visited):
                    if v==1000:
                        env[Agent_list[agnt_i].goals[i][0]][Agent_list[agnt_i].goals[i][1]] = d[VSTD_P] # sets visited point to white           
                
                
                env[Agent_list[agnt_i].x][Agent_list[agnt_i].y] = d[UAV_P]  # sets the agent tile to blue
            
            for k in range(len(NFZ_list)):
                env[NFZ_list[k].x][NFZ_list[k].y] = d[NFZ_P]  # sets the no fly location to red
            img = Image.fromarray(env, 'RGB')  # reading to rgb
            img = img.resize((300, 300), resample=Image.BOX)  
            cv2.imshow("Q", np.array(img))
            
            
            for agnt_i in range(len(Agent_list)):
                if reward_each_agent[agnt_i] == POI_REWARD or reward_each_agent[agnt_i] == -NFZ_PENALTY:  # crummy code to hang at the end if we reach abrupt end for good reasons or not.
                    if reward_each_agent[agnt_i] == POI_REWARD:
                            Agent_list[agnt_i].visited[Agent_list[agnt_i].idx] = 1000
                            

                            val, idx = min((val, idx) for (idx, val) in enumerate(Agent_list[agnt_i].visited))

                            if val==1000:
                                print("Agent ", agnt_i, "Completed : ", Agent_list[agnt_i].visited)
                                break

                            print("Agent ", agnt_i, "Visited : ", Agent_list[agnt_i].visited)
                            Agent_list[agnt_i].val = val
                            Agent_list[agnt_i].idx = idx


                            current_PoI_list[agnt_i].x = Agent_list[agnt_i].goals[idx][0]
                            current_PoI_list[agnt_i].y = Agent_list[agnt_i].goals[idx][1]
                            continue

                    if cv2.waitKey(500) & 0xFF == ord('q'):
                        break
                else:
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                    
        episode_reward += reward_each_agent[agnt_i] 
        for agnt_i in range(len(Agent_list)):
            if reward_each_agent[agnt_i] == POI_REWARD or reward_each_agent[agnt_i] == -NFZ_PENALTY:
                if reward_each_agent[agnt_i] == POI_REWARD:
                    Agent_list[agnt_i].visited[Agent_list[agnt_i].idx] = 1000

                    val, idx = min((val, idx) for (idx, val) in enumerate(Agent_list[agnt_i].visited))

                    if val==1000:
                        break

                    Agent_list[agnt_i].val = val
                    Agent_list[agnt_i].idx = idx


                    current_PoI_list[agnt_i].x = Agent_list[agnt_i].goals[idx][0]
                    current_PoI_list[agnt_i].y = Agent_list[agnt_i].goals[idx][1]
                    continue
                break
    p=0           
    for agent_i in range(len(Agent_list)):
        for k in range(len(Agent_list[agent_i].visited)):
            p += Agent_list[agent_i].visited[k]
            
    if p>MAX_VISITED:
        MAX_VISITED=p
        show_it=paths_of_all_agents
        temp = []
        for i in range(len(Agent_list)):
            temp.append(Agent_list[i].goals)
        
        rewards = temp

    episode_rewards.append(episode_reward)
    epsilon *= EPS_DECAY

    
cv2.destroyAllWindows()
moving_avg = np.convolve(episode_rewards, np.ones((SHOW_EVERY,))/SHOW_EVERY, mode='valid')

with open(f"qtable-{int(time.time())}.pickle", "wb") as f:
    pickle.dump(q_table, f)

###########################################################


on #0, epsilon is 0.5
1000 ep mean: nan
Agent  0 Visited :  [17.46424919657298, 12.041594578792296, 1000, 19.924858845171276, 16.492422502470642, 15.524174696260024, 13.45362404707371]


C:\Users\alvia\anaconda3\envs\WorkEnv\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\alvia\anaconda3\envs\WorkEnv\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Agent  1 Visited :  [8.602325267042627, 8.06225774829855, 5.0, 8.48528137423857, 1000, 5.0, 9.899494936611665]
###########################################################


on #1000, epsilon is 0.4524164467792781
1000 ep mean: -302.336
Agent  0 Visited :  [11.180339887498949, 13.152946437965905, 12.041594578792296, 7.615773105863909, 13.038404810405298, 1000, 5.0]
Agent  0 Visited :  [11.180339887498949, 13.152946437965905, 12.041594578792296, 1000, 13.038404810405298, 1000, 1000]
Agent  1 Visited :  [16.76305461424021, 12.649110640673518, 11.045361017187261, 9.055385138137417, 9.0, 9.848857801796104, 1000]
###########################################################


on #2000, epsilon is 0.40936128263277455
1000 ep mean: -294.809
Agent  1 Visited :  [1000, 8.06225774829855, 16.278820596099706, 3.605551275463989, 10.63014581273465, 12.206555615733702, 11.045361017187261]
Agent  0 Visited :  [10.63014581273465, 7.615773105863909, 6.082762530298219, 13.038404810405298, 3.0, 1000, 4.47213

Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
###########################################################


on #12000, epsilon is 0.15058806979843486
1000 ep mean: -242.445
Agent  0 Visited :  [6.0, 4.123105625617661, 21.2602916254693, 13.601470508735444, 1000, 7.615773105863909, 16.55294535724685]
Agent  2 Visited :  [10.770329614269007, 6.708203932499369, 8.94427190999916, 1000, 8.246211251235321, 8.54400374531753]
Agent  0 Visited :  [1000, 1000, 21.2602916254693, 13.601470508735444, 1000, 7.615773105863909, 16.55294535724685]
Agent  0 Visited :  [1000, 1000, 21.2602916254693, 1000, 1000, 1000, 16.55294535724685]
Agent  1 

Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [10

Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
###########################################################


on #20000, epsilon is 0.06766087474137944
1000 ep mean: -158.179
Agent  1 Visited :  [4.123105625617661, 11.313708498984761, 6.708203932499369, 11.180339887498949, 10.04987562112089, 5.830951894845301, 1000]
Agent  1 Visited :  [1000, 11.313708498984761, 6.708203932499369, 11.180339887498949, 10.04987562112089, 1000, 1000]
Agent  1 Visited :  [1000, 11.313708498984761, 1000, 11.180339887498949, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Ag

Agent  1 Visited :  [11.40175425099138, 1000, 10.63014581273465, 1000, 11.180339887498949, 1000, 10.0]
Agent  0 Visited :  [11.40175425099138, 17.08800749063506, 1000, 1000, 12.165525060596439, 11.0, 1000]
Agent  0 Visited :  [1000, 17.08800749063506, 1000, 1000, 12.165525060596439, 1000, 1000]
Agent  2 Visited :  [9.433981132056603, 1000, 1000, 10.0, 1000, 9.055385138137417]
###########################################################


on #22000, epsilon is 0.05539548493637325
1000 ep mean: -85.015
Agent  1 Visited :  [20.0, 11.313708498984761, 19.313207915827967, 5.0, 1000, 7.0, 16.401219466856727]
Agent  1 Visited :  [20.0, 11.313708498984761, 19.313207915827967, 1000, 1000, 1000, 16.401219466856727]
Agent  0 Visited :  [8.54400374531753, 1000, 12.529964086141668, 3.605551275463989, 17.804493814764857, 15.231546211727817, 5.0]
Agent  0 Visited :  [8.54400374531753, 1000, 12.529964086141668, 1000, 17.804493814764857, 15.231546211727817, 1000]
Agent  1 Visited :  [20.0, 1000, 19.31320

In [8]:
import time
ts = str(time.time())

for i in range(len(show_it)):
    create_output_files(show_it[i], "Q_"+ts+"_UAV"+str(i)+".txt")
    create_output_files(rewards[i], "Q_"+ts+"_UAV"+str(i)+"_R"+".txt")
    
create_output_files(rewards[i], "Q_"+ts+"_NFZ"+".txt")